In [1]:
#from whisper_jax import FlaxWhisperForConditionalGeneration
# from transformers import WhisperForConditionalGeneration
import torch
from functions import *
from functools import partial
import math
import evaluate
from transformers import Wav2Vec2ProcessorWithLM,Wav2Vec2Processor
from pyctcdecode import build_ctcdecoder
import warnings
warnings.filterwarnings('ignore')

IsTPU = False
batch_size = 16
device = 'cuda'
dtype = torch.float32
speech_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation_files/'
data_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/dlsprint/validation.csv'
add_audio2 = '/home/zhenlan/Desktop/Projects/Bengali ASR/RESPIN/dev/'
model_path = '/home/zhenlan/Desktop/Projects/Bengali ASR/IndicFT'
speech_path0 = 'data/train_mp3s/'
data_path0 = 'data/train.csv'
num_workers = 8
output_path = ''

In [2]:
pad_to_multiple_of = 1
max_length_gen = 24

processor = Wav2Vec2ProcessorWithLM.from_pretrained("/home/zhenlan/Desktop/Projects/Bengali ASR/arijitx-wav2vec2-xls-r-300m-bengali")
# processor = Wav2Vec2ProcessorWithLM.from_pretrained("/home/zhenlan/Desktop/Projects/Bengali ASR/YellowKing_processor")
# processor = Wav2Vec2Processor.from_pretrained(model_path)
tokenizer = processor.tokenizer
feature_extractor = processor.feature_extractor
text = pd.read_csv(data_path)
text0 = pd.read_csv(data_path0)

In [3]:
# vocab_dict = processor.tokenizer.get_vocab()
# sorted_vocab_dict = {k: v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}
# decoder = build_ctcdecoder(
#     list(sorted_vocab_dict.keys()),
#     kenlm_model_path='/home/zhenlan/Desktop/Projects/Bengali ASR/arijitx-wav2vec2-xls-r-300m-bengali/language_model/5gram.bin',
#     # unigrams = '/home/zhenlan/Desktop/Projects/Bengali ASR/arijitx-wav2vec2-xls-r-300m-bengali/language_model/unigrams.txt',
#     # alpha=0.5,  # tuned on a val set
#     # beta=1.0,  # tuned on a val set
# )
# processor_with_lm = Wav2Vec2ProcessorWithLM(
#     feature_extractor=processor.feature_extractor,
#     tokenizer=processor.tokenizer,
#     decoder=decoder
# )

In [4]:
df = pd.read_csv('text',names=['code'])
df[['code','sentence']] = df["code"].str.split(" ", n=1, expand=True)
df.sentence = df.sentence.str.strip()

In [5]:
dataset0 = AudioDataset(text0,speech_path0,lambda x:x.id+'.mp3',\
                        orig_sr=32000, target_sr=16000)
train_loader_CE0 = DataLoader(dataset0, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor))
train_loader_WER0 = DataLoader(dataset0, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor,IsTrain=False))

dataset1 = AudioDataset(text,speech_path,lambda x:x.path,\
                       None,orig_sr=32000, target_sr=16000)
train_loader_CE = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor))
train_loader_WER = DataLoader(dataset1, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor,IsTrain=False))

dataset2 = AudioDataset(df,add_audio2,\
                        lambda x:x.code.split('_')[-1] + '.wav',orig_sr=16000, target_sr=16000)
train_loader_CE2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor))
train_loader_WER2 = DataLoader(dataset2, batch_size=batch_size, shuffle=False, num_workers=num_workers, \
                         collate_fn=partial(collate_fn_pt_wav2vec,tokenizer=tokenizer,feature_extractor=feature_extractor,IsTrain=False))

In [6]:
from transformers import Wav2Vec2ForCTC
# model = Wav2Vec2ForCTC.from_pretrained("Sameen53/training_45k").to('cuda')
# model = Wav2Vec2ForCTC.from_pretrained("/home/zhenlan/Desktop/Projects/Bengali ASR/YellowKing_model").to('cuda')
model = torch.load('/home/zhenlan/Desktop/Projects/Bengali ASR/wav2vec_CTC/wav2vec.pth').to('cuda')
# model = Wav2Vec2ForCTC.from_pretrained(model_path).to('cuda')
model.eval();

In [7]:
from transformers import pipeline
my_asrLM = pipeline("automatic-speech-recognition", \
                    model=model ,\
                    feature_extractor =processor.feature_extractor, \
                    tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)
# audio,attention_mask,txt = next(iter(train_loader_WER0))
# out = my_asrLM(list(audio))
metric = evaluate.load("wer")
def metric_one_step(audio,attention_mask,txt,beam_width=100):
    transcriptions = [item['text'] for item in my_asrLM(list(audio))]
    wer = metric.compute(predictions=transcriptions, references=txt)
    return wer

In [8]:
# metric = evaluate.load("wer")
# def metric_one_step(audio,attention_mask,txt,beam_width=100):
#     audio,attention_mask = torch.tensor(audio,device=device),\
#                                  torch.tensor(attention_mask,device=device)
#     with torch.no_grad():
#         logits = model(audio,attention_mask=attention_mask).logits.cpu().numpy()
    
#     transcriptions = [processor_with_lm.decode(logit,beam_width=beam_width).text for logit in logits]
#     wer = metric.compute(predictions=transcriptions, references=txt)
#     return wer

def eval_one_step(audio,input_ids,attention_mask):
    audio,input_ids,attention_mask = torch.tensor(audio,device=device),\
                                 torch.tensor(input_ids,device=device),\
                                 torch.tensor(attention_mask,device=device)
    with torch.no_grad():
        loss = model(audio,attention_mask=attention_mask,labels=input_ids).loss
        return loss

DATASET0

In [9]:
import logging
logging.basicConfig(filename='performance/FTCTC0827_performance.log', filemode='w', level=logging.INFO,
                    format='%(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [10]:
%%time
# # model_all2
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader_CE0):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(audio,input_ids,attention_mask)
    if math.isinf(l.item()) or math.isnan(l.item()):
        test_loss += 5.0 # CTC loss overflow
    else:
        test_loss += l.item()
    if j>60:
        break
test_loss /= (j+1)
logging.info(f"CE loss on data0: {test_loss:.4f}")

CPU times: user 10.2 s, sys: 1.27 s, total: 11.5 s
Wall time: 12.2 s


In [11]:
# %%time
# model_all2
test_loss = 0
for j,(audio,attention_mask,txt) in enumerate(train_loader_WER0):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,attention_mask,txt)
    test_loss += l
    if j>60:
        break
test_loss /= (j+1)
logging.info(f"WER loss on data0: {test_loss:.4f}")

DATASET1

In [12]:
%%time
# # model_all2
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader_CE):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(audio,input_ids,attention_mask)
    if math.isinf(l.item()) or math.isnan(l.item()):
        test_loss += 5.0 # CTC loss overflow
    else:
        test_loss += l.item()
    if j>60:
        break
test_loss /= (j+1)
logging.info(f"CE loss on data1: {test_loss:.4f}")

CPU times: user 10.3 s, sys: 681 ms, total: 10.9 s
Wall time: 11.7 s


In [13]:
%%time
# model_all2
test_loss = 0
for j,(audio,attention_mask,txt) in enumerate(train_loader_WER):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,attention_mask,txt)
    test_loss += l
    if j>60:
        break
test_loss /= (j+1)
logging.info(f"WER loss on data1: {test_loss:.4f}")

CPU times: user 3min 17s, sys: 1.23 s, total: 3min 18s
Wall time: 39.2 s


DATASET2

In [14]:
%%time
# # model_all2
test_loss = 0
for j,(audio,input_ids,attention_mask) in enumerate(train_loader_CE2):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = eval_one_step(audio,input_ids,attention_mask)
    if math.isinf(l.item()) or math.isnan(l.item()):
        test_loss += 5.0 # CTC loss overflow
    else:
        test_loss += l.item()
    if j>60:
        break
test_loss /= (j+1)
logging.info(f"CE loss on data2: {test_loss:.4f}")

CPU times: user 20.8 s, sys: 1.16 s, total: 22 s
Wall time: 23.1 s


In [15]:
%%time
# model_all2
test_loss = 0
for j,(audio,attention_mask,txt) in enumerate(train_loader_WER2):
    #audio,input_ids,attention_mask = jnp.array(audio,dtype=dtype),jnp.array(input_ids),jnp.array(attention_mask)
    l = metric_one_step(audio,attention_mask,txt)
    test_loss += l
    if j>60:
        break
test_loss /= (j+1)
logging.info(f"WER loss on data2: {test_loss:.4f}")

CPU times: user 3min 30s, sys: 1.55 s, total: 3min 32s
Wall time: 55.9 s
